In [ ]:
import os
import pandas as pd
# set pandas column width
pd.set_option('display.max_colwidth', 200)

import openvalidators
from utils import download_data, get_runs
import plotting 


# Download the wandb Data

In [ ]:
DEFAULT_FILTERS = {"tags": {"$in": [openvalidators.__version__]}}

# download most recent runs from wandb
runs = get_runs(filters=DEFAULT_FILTERS, return_paths=True)
runs = runs[:30]
print('\nDownloading data for the following runs:')
for i, run in enumerate(runs, start=1):
    print(f'{i:>3} - {run}')
    
df = download_data(runs) 

# Print some info about the dataframe
df.info()
print(f'History for {len(runs)} runs contains {len(df)} records. Dataframe shape: {df.shape}')

In [ ]:
# Contents of first log entry
df.iloc[0]

In [ ]:
# Get length of downloaded runs
df.groupby('run_id').size()

# Network Health Overview
The plot below shows the diversity of completions produced by each UID, which is defined as the number of unique completions divided by the total number of completions. A low diversity score indicates that the miner is producing the same completions over and over again, while a high diversity score indicates that the miner is producing a wide variety of completions. Good UIDs should be in the top right corner. Color indicates average reward for that UID

In [ ]:

plotting.plot_uid_diversty(df, remove_unsuccessful=True)

# Logging throughput
The plot below shows the event logging rate

In [ ]:
plotting.plot_throughput(df,n_minutes=10)

# Dendrite Response Success Rates
The plots below show the success rates of the dendrite responses. The success rate is defined as the number of completions that recieve nonzero reward.

In [ ]:
# plots dendrite completion success rates (for answers by default), default is 20 most queried uids
plotting.plot_dendrite_rates(df)


In [ ]:
# plot dendrite completion rates for specific UID followups 
plotting.plot_dendrite_rates(df, uid_col='followup_uids', reward_col='followup_rewards', uids=[11,4,72,105,21,444,51,34,100,200,299])


# Analyze Completion Rates
The plots below show the rate of popular completions produced by the network. These can help to detect spam/attacks.

**Note**: Empty string responses are a result of unsuccessful completions.

In [ ]:
# Show rate of 10 most popular followup completions 
plotting.plot_completion_rates(df, msg_col='followup_completions', time_interval='H', ntop=10).update_layout(yaxis_type='log')

In [ ]:
# same, but for answers
plotting.plot_completion_rates(df, msg_col='answer_completions', time_interval='H', ntop=10).update_layout(yaxis_type='log')

In [ ]:
# regex-based completion search
plotting.plot_completion_rates(df, msg_col='answer_completions', time_interval='H', ntop=5, completion_regex='Beyoncé')

## Rewards for a Set of Completions
The plots below show the rewards for a set of completions. This is a complement to the completion rates, as they show whether the completions are actually rewarded highly or not.

In [ ]:
# plot reward rates for 5 most popular followup completions
plotting.plot_completion_rewards(df, ntop=5, msg_col='followup_completions', reward_col='followup_rewards', uid_col='followup_uids')


In [ ]:
# same, but for answers
plotting.plot_completion_rewards(df, ntop=5, msg_col='answer_completions', reward_col='answer_rewards', uid_col='answer_uids')


# Leaderboards
The plots below show the leaderboard of UIDs and completions, based on various factors.

In [ ]:
# Get UIDs with highest average answer rewards 
plotting.plot_leaderboard(df, ntop=30)

In [ ]:
# Get answer completions with highest average rewards 
plotting.plot_leaderboard(df, ntop=10, group_on='answer_completions', agg_col='answer_rewards', alias=True)

In [ ]:
# Get followup completions with highest max reward 
plotting.plot_leaderboard(df, ntop=10, group_on='followup_completions', agg_col='followup_rewards', agg='max', alias=True)

In [ ]:
# Get answer completions with highest frequency
plotting.plot_leaderboard(df, ntop=5, group_on='answer_completions', agg_col='answer_rewards', agg='size', alias=True)